# Doc2Vecの実行条件調査（part3）

- ラベルごとの教師データを最大３０件とします（このためテストデータから学習セットの一部が失われます）
 

- 品詞を落とさないようにする


- パラメータを一部変更（DBOWを使用／feature数を調整）


accuracy=0.767 とかなりのダウンが確認されました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/18 PM 02:34:37 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/18 PM 02:34:37 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

#### ラベル毎の教師データ数を制限します。

In [6]:
answer_id_count = {}

selected_answer_ids = []
selected_questions = []
selected_separated_sentences = []

sample_limit_count = 30

for index, answer_id in enumerate(answer_ids):
    if answer_id not in answer_id_count.keys():
        answer_id_count[answer_id] = 0

    if answer_id_count[answer_id] < sample_limit_count:
        '''
            データをリストに格納
        '''
        selected_answer_ids.append('%04d_%02d' % (answer_id, answer_id_count[answer_id]))
        selected_questions.append(questions[index])
        selected_separated_sentences.append(_separated_sentences[index])
        answer_id_count[answer_id] += 1

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [7]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    教師データ件数＝4992
'''
sentences = corpus_to_sentences(selected_separated_sentences, selected_answer_ids)
sentence_list = list(sentences)
len(sentence_list)

4992

In [9]:
'''
    パラメータ変更点：
    　学習時にDBOWを使用する
    　featureの数をサンプル件数に近い値に設定
    　反復回数をfeature数の10倍に設定
'''
n_feature = round(len(sentence_list) + 49, -2) # 切り上げし、100の倍数に変更
n_iter = n_feature * 10
print('Doc2Vec: feature(size)=%d, iter=%d' % (n_feature, n_iter))

model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)

Doc2Vec: feature(size)=5000, iter=50000


In [10]:
'''
    ボキャブラリ生成／学習実行
    学習モデルは、ファイルに保存しておく
'''
model.build_vocab(sentence_list)
model.train(sentence_list)

model_path = 'prototype/better_algorithm/doc2vec_30.model'
model.save(model_path)

In [11]:
'''
    モデル内に保持されているベクトルの数を取得
    （教師データ数と同じであることを確認）
'''
len(model.docvecs)

4992

In [12]:
model.docvecs

### (2-4) 予測処理

質問文は一部「揺れ」を与えたものとなっております

In [13]:
model_path = 'prototype/better_algorithm/doc2vec_30.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [14]:
'''
    マウスが破損（正解＝4458）
'''
predict('マウスが破損', model_path)

(['マウス', 'が', '破損'],
 [('4458_19', 0.767440915107727),
  ('4530_12', 0.7669717073440552),
  ('4458_00', 0.766453742980957),
  ('4530_21', 0.7648692727088928),
  ('4564_01', 0.6634461879730225),
  ('4581_03', 0.6539479494094849),
  ('4581_01', 0.6526361703872681),
  ('4480_01', 0.6298810839653015),
  ('4480_15', 0.6294967532157898),
  ('4480_04', 0.6290603876113892)])

In [15]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', 'を', '使用', 'する', 'たい'],
 [('4516_00', 0.7407429218292236),
  ('4516_29', 0.6707862615585327),
  ('4521_00', 0.6437191367149353),
  ('4557_07', 0.6104491949081421),
  ('4602_19', 0.6051332950592041),
  ('4499_24', 0.6024231910705566),
  ('4602_26', 0.600010335445404),
  ('4429_26', 0.5950993895530701),
  ('4429_00', 0.5862820148468018),
  ('4499_19', 0.5855737924575806)])

In [16]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'の', 'アドレス'],
 [('7040_10', 0.723394513130188),
  ('7040_01', 0.7212193012237549),
  ('7040_04', 0.6775554418563843),
  ('7040_13', 0.6774689555168152),
  ('7040_12', 0.6187596917152405),
  ('7040_03', 0.6170516014099121),
  ('7040_00', 0.6078429818153381),
  ('7040_02', 0.5907718539237976),
  ('7040_19', 0.5801676511764526),
  ('7040_11', 0.5713045597076416)])

In [17]:
'''
    Office2010を使用したいのですが（正解＝4625）
'''
predict('Office2010を使用したいのですが', model_path)

(['Ｏｆｆｉｃｅ', '２０１０', 'を', '使用', 'する', 'たい', 'の', 'です', 'が'],
 [('4625_03', 0.7581274509429932),
  ('4625_08', 0.7503246068954468),
  ('4625_28', 0.6857753396034241),
  ('4625_11', 0.6822872161865234),
  ('4625_09', 0.6761207580566406),
  ('4625_27', 0.6539368629455566),
  ('4625_02', 0.6305592656135559),
  ('4523_02', 0.6277899742126465),
  ('4523_24', 0.6275372505187988),
  ('4625_29', 0.6248303055763245)])

In [18]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？
    （正解＝4504だが、学習時の教師データから抜け落ちたもの）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯',
  'から',
  'サイボウズ',
  'を',
  '使う',
  'たい',
  'の',
  'です',
  'が',
  'どう',
  'する',
  'た',
  '出来る',
  'ます',
  'か'],
 [('4541_07', 0.6443217396736145),
  ('4501_06', 0.5898873805999756),
  ('4594_16', 0.5832247734069824),
  ('4501_15', 0.5617283582687378),
  ('4501_27', 0.552489161491394),
  ('4533_28', 0.5478017330169678),
  ('4541_04', 0.5394222140312195),
  ('4543_18', 0.537372887134552),
  ('4541_06', 0.535804033279419),
  ('4501_00', 0.523921549320221)])

## (3) accuracy 測定

ここからが本題です

In [19]:
model_path = 'prototype/better_algorithm/doc2vec.model'

len(selected_separated_sentences)

4992

In [20]:
def predict_similarity(separated_sentence, model_path):
    corpus = separated_sentence.split()
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

In [21]:
def get_prediction_statistics(separated_sentences, answer_ids, model_path):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i], model_path)
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [22]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids, model_path)

In [23]:
ncorrect_by_corpus_len = {}
nsample_by_corpus_len = {}

ncorrect = 0
nsample = 0

'''
    予測結果を、質問文の単語数毎／回答ID毎に統計する
'''
for statistics in prediction_statistics:
    i, corpus_len, preferred_answer_id, answer_id, similarity = statistics
    
    '''
        質問文の単語数ごとに統計を取る
    '''
    if corpus_len not in nsample_by_corpus_len.keys():
        ncorrect_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] = 0
    nsample_by_corpus_len[corpus_len] += 1
    
    '''
        正解かどうか検査
        （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
    '''
    nsample += 1
    if preferred_answer_id[0:4] == answer_id[0:4]:
        ncorrect += 1
        ncorrect_by_corpus_len[corpus_len] += 1

In [24]:
'''
    質問文の単語数ごとの統計情報を編集
'''
info_by_corpus_len = []
for k, v in ncorrect_by_corpus_len.items():
    info_by_corpus_len.append((
        k, 
        ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
        ncorrect_by_corpus_len[k], 
        nsample_by_corpus_len[k]
    ))

In [25]:
'''
    全体の正解率
'''
print("accuracy=%0.3f (%d/%d)" % (
    ncorrect/nsample, ncorrect, nsample))

accuracy=0.767 (3828/4992)


In [26]:
'''
    質問文の単語数ごとの正解率をリスト
'''
for info in info_by_corpus_len:
    print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
        info[0], info[1], info[2], info[3]
    ))

word_count= 1: accuracy=0.667 (4/6)
word_count= 2: accuracy=0.838 (155/185)
word_count= 3: accuracy=0.758 (379/500)
word_count= 4: accuracy=0.743 (491/661)
word_count= 5: accuracy=0.728 (429/589)
word_count= 6: accuracy=0.722 (366/507)
word_count= 7: accuracy=0.757 (346/457)
word_count= 8: accuracy=0.758 (298/393)
word_count= 9: accuracy=0.788 (242/307)
word_count=10: accuracy=0.803 (212/264)
word_count=11: accuracy=0.748 (163/218)
word_count=12: accuracy=0.790 (143/181)
word_count=13: accuracy=0.874 (167/191)
word_count=14: accuracy=0.841 (106/126)
word_count=15: accuracy=0.775 (107/138)
word_count=16: accuracy=0.747 (56/75)
word_count=17: accuracy=0.770 (47/61)
word_count=18: accuracy=0.875 (28/32)
word_count=19: accuracy=0.824 (14/17)
word_count=20: accuracy=0.864 (19/22)
word_count=21: accuracy=0.875 (14/16)
word_count=22: accuracy=0.714 (5/7)
word_count=23: accuracy=1.000 (5/5)
word_count=24: accuracy=0.750 (6/8)
word_count=26: accuracy=1.000 (2/2)
word_count=27: accuracy=1.000 (3